<h1><p style="display: block; text-align: center;">Python Libraries for GIS and Jupyter Notebook as a Mapping Application</p></h1>

<h3><p style="display: block; text-align: center;">WA Women in GIS and Technology - Wed March 6, 2019</p></h3>

<h5><p style="display: block; text-align: center;">Christy Heaton</p></h5>

<h1><p style="display: block; text-align: center;">About me</p></h1>

![title](img/FS_bio.PNG) 

![title](img/maptimeseattle_website.PNG) 

Find us on [Meetup!](https://www.meetup.com/MaptimeSEA)

# Outline

### Open Source vs. Proprietary

### Python

### Python and Esri

### Jupyter Notebook

### Fun Example

# [Open Source vs. Proprietary](https://opensource.com/resources/what-open-source)

# [Python](https://www.python.org/about/)

# Python and Esri

# [Jupyter Notebook](https://jupyter.org/)

## This presentation is running inside of a Jupyter Notebook

In [ ]:
import platform
print('Using Python version', platform.python_version())

# Example using Python, GIS Libraries, and Jupyter

# Spatial Problem: In what cities will we be able to see upcoming solar eclipses?

![title](img/Christy_eclipse.jpg)

![title](img/_alignment-lines-720.png)

![title](img/Corona-2017-Aug-21-m.jpg)

### We can use Python and Jupyter to find out:

1.  When and where will we be able to see an eclipse?

2.  Which upcoming eclipse will pass over the most people?

3.  What are the largest cities in that eclipse?


# Python Mapping Libraries

## [Matplotlib](https://matplotlib.org/)

#### A Python plotting library which produces publication quality maps and diagrams in both static and interactive formats. 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

## [Pandas](https://pandas.pydata.org/)

#### Provides high-performance, easy-to-use data structures and data analysis tools

## [GeoPandas](http://geopandas.org/)

#### Geographically-enabled Pandas, depends on [Shapely](https://github.com/Toblerity/Shapely) for manipulation and analysis of planar geometric objects

In [ ]:
import pandas as pd
import geopandas as gpd

# Let's Start by Making a Simple Map

In [ ]:
# GeoPandas comes with some data that we can quickly load in
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
# And check out the top few rows - notice it comes with attributes like population and GDP, 
# as well as geometry 
world.head()

In [ ]:
world.crs

In [ ]:
# Use the plot method to view the spatial component
world.plot()

In [ ]:
# You'll usually want to customize how your map looks
world.plot(color='grey', linewidth=0.5, edgecolor='white', figsize=(15,10))

In [ ]:
# We'll need to load in some local data
import os
data_pth = "../Data/" 

In [ ]:
# GeoPandas comes with city data, but its just the capital cities, so I'll load in my own
cities = gpd.read_file(os.path.join(data_pth, "ne_10m_populated_places.shp"))

In [ ]:
# View the top few rows
cities.head()

In [ ]:
cities.crs

In [ ]:
# And plot to view the spatial component
cities.plot(figsize=(15,10), color='orange', markersize=5)

In [ ]:
world.crs == cities.crs

In [ ]:
base = world.plot(color='grey', linewidth=0.5, edgecolor='white', figsize=(15,10))
cities.plot(ax=base, color='orange', markersize=5)
base.set_axis_off() 

### What do we want to know?

1.  When and where will we be able to see an eclipse?


In [ ]:
# Load in Eclipse data
eclipses = gpd.read_file(os.path.join(data_pth, "Eclipses.shp"))

In [ ]:
# Examine the first few rows
eclipses.head()

In [ ]:
# Check the coorindate reference system
eclipses.crs

In [ ]:
# And let's plot with a customized style
eclipses.plot(figsize=(15,10), color='black', edgecolor='yellow', alpha=0.75)

In [ ]:
# GeoPandas let's you plot the minimum bounding box of each entity
eclipses.envelope.plot(figsize=(15,10))

In [ ]:
# Let's plot the eclipses on top of our world data
base = world.plot(color='lightgrey', linewidth=0.5, edgecolor='white', figsize=(15,10))
eclipses.plot(ax=base, color='black', edgecolor='yellow', alpha=0.75)
base.set_axis_off()

In [ ]:
# But how do we know which is which?
base = world.plot(color='lightgrey', linewidth=0.5, edgecolor='white', figsize=(15,10))

# We can add a catetorical value and set the column to Year. We can also add a legend.
eclipses.plot(ax=base, cmap='tab10', alpha=0.5, categorical = True, column = "Year", legend=True)
base.set_axis_off()
base.get_legend().set_bbox_to_anchor((.05,.7))

### What do we want to know?

1.  When and where will we be able to see an eclipse?

2.  What upcoming eclipse will pass over the most people?

In [ ]:
# Is the cities data still in the same crs as the eclipse data? It needs to be to plot correctly!
world.crs == eclipses.crs == cities.crs

In [ ]:
# Now we'll plot our basemap, our eclipse paths, and our cities
base = world.plot(color='lightgrey', linewidth=0.5, edgecolor='white', figsize=(15,10))
eclipses.plot(ax=base, color='black', edgecolor='yellow', alpha=0.75)
cities.plot(ax=base, color='orange', markersize=5)
base.set_axis_off()

In [ ]:
# Start by getting all the cities that intersect any eclipse, a spatial join using the geom column
allecities = gpd.sjoin(cities, eclipses, how='inner', op='intersects')

In [ ]:
allecities.head()

In [ ]:
allecities = pd.DataFrame(allecities[['POP', 'Year']])

In [ ]:
allecities.head()

In [ ]:
# Create a temporary variable to hold our years and populations
temp_group = allecities.groupby(['Year'])[["POP"]].sum()

In [ ]:
temp_group

In [ ]:
# Add that column to our eclipse data, joining on the Year column
eclipses_pop = pd.merge(eclipses, temp_group, left_on='Year', right_index=True)

In [ ]:
# Now we have populations associated with each eclipse
eclipses_pop.head()

In [ ]:
# Sort the results to find out which eclipse will cover the most people
eclipses_pop.sort_values('POP', ascending = False)

In [ ]:
# Let's plot this
base = world.plot(color='lightgrey', linewidth=0.5, edgecolor='white', figsize=(15,10))
eclipses_pop.plot(ax=base, cmap='Oranges', alpha=0.5, column = "POP")
base.set_axis_off()

In [ ]:
# As we saw earlier, this eclipse is happening in 2024
base = world.plot(color='lightgrey', linewidth=0.5, edgecolor='white', figsize=(15,10))
eclipses.plot(ax=base, cmap='tab10', alpha=0.5, categorical = True, column = "Year", legend=True)
base.set_axis_off()
base.get_legend().set_bbox_to_anchor((.05,.7))

### What do we want to know?

1.  When and where will we be able to see an eclipse?

2.  What upcoming eclipse will pass over the most people?

3.  What are the largest cities in that eclipse?


In [ ]:
# Let's assign the 2024 eclipse to its own variable
myeclipse = eclipses[(eclipses['Year'] == 2024)]

In [ ]:
# Let's plot it 
base = world.plot(color='lightgrey', linewidth=0.5, edgecolor='white', figsize=(15,10))
myeclipse.plot(ax=base, color='black', edgecolor='yellow', alpha=0.75)
cities.plot(marker='*', color='orange', markersize=5, ax=base)
base.set_axis_off()

In [ ]:
# Plot again, but this time I want to zoom in on that eclipse path
base = world.plot(color='lightgrey', linewidth=0.5, edgecolor='white', figsize=(15,10))
myeclipse.plot(ax=base, color='black', edgecolor='yellow', alpha=0.75)
cities.plot(marker='*', color='orange', markersize=5, ax=base)
bounds = myeclipse.geometry.bounds
plt.xlim([bounds.minx.min()-5, bounds.maxx.max()+5])
plt.ylim([bounds.miny.min()-5, bounds.maxy.max()+5])
base.set_axis_off()

In [ ]:
# Let's style the plot so that the eclipse looks eclipse-ier
base = world.plot(color='lightgray', linewidth=0.5, edgecolor='white', figsize=(15,10))
myeclipse.plot(ax=base, color='black', edgecolor='yellow', alpha=0.75)
cities.plot(ax=base, color='black', markersize=20, edgecolor='white')
bounds = myeclipse.geometry.bounds
plt.xlim([bounds.minx.min()-5, bounds.maxx.max()+5])
plt.ylim([bounds.miny.min()-5, bounds.maxy.max()+5])
base.set_axis_off()

In [ ]:
# We can see that there are quite a few cities that intersect my path! But we want to know which ones.
# Do a spatial join to get the intersection
ecities = gpd.sjoin(cities, myeclipse, how='inner', op='intersects')

In [ ]:
# Let's plot the results!
base = world.plot(color='lightgray', linewidth=0.5, edgecolor='white', figsize=(15,10))
myeclipse.plot(ax=base, color='black', edgecolor='yellow', alpha=0.75)
ecities.plot(ax=base, color='black', markersize=20, edgecolor='white')
bounds = myeclipse.geometry.bounds
plt.xlim([bounds.minx.min()-5, bounds.maxx.max()+5])
plt.ylim([bounds.miny.min()-5, bounds.maxy.max()+5])
base.set_axis_off()

In [ ]:
base = world.plot(color='lightgray', linewidth=0.5, edgecolor='white', figsize=(15,10))
myeclipse.plot(ax=base, color='black', edgecolor='yellow', alpha=0.75)
ecities.plot(ax=base, color='black', edgecolor='white', markersize=cities['POP']/1000)
bounds = myeclipse.geometry.bounds
plt.xlim([bounds.minx.min()-5, bounds.maxx.max()+5])
plt.ylim([bounds.miny.min()-5, bounds.maxy.max()+5])
base.set_axis_off()

In [ ]:
print('Wow! There are {} cities in the eclipse path!'.format(len(ecities)))
print("Here are the largest 5:")
print(ecities.sort_values('POP', ascending = False)['NAME'].head(5))

In [ ]:
top_five = ecities.sort_values('POP', ascending = False)[:5]

In [ ]:
base = world.plot(color='lightgray', linewidth=0.5, edgecolor='white', figsize=(15,10))
myeclipse.plot(ax=base, color='black', edgecolor='yellow', alpha=0.25)
top_five.plot(ax=base, color='black', edgecolor='white', markersize=cities['POP']/400)
for x, y, label in zip(top_five.geometry.x, top_five.geometry.y, top_five.NAME):
    base.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")
bounds = myeclipse.geometry.bounds
plt.xlim([bounds.minx.min()-5, bounds.maxx.max()+5])
plt.ylim([bounds.miny.min()-5, bounds.maxy.max()+5])
base.set_axis_off()

# Bonus: Make it Slippy

## [Folium](https://python-visualization.github.io/folium/)

#### Binds the power of Python with leaflet.js

In [ ]:
import folium

In [ ]:
Seattle = [47.61, -122.33]

osm_map = folium.Map(location=Seattle, zoom_start=10)
osm_map

In [ ]:
map_stamen_toner = folium.Map(
    location=Seattle,
    tiles='stamentoner',
    zoom_start=10
)
map_stamen_toner

In [ ]:
map_stamen_water = folium.Map(
    location=Seattle,
    tiles='stamenwatercolor',
    zoom_start=10
)
map_stamen_water

In [ ]:
# Let's find the centroid of the eclipse we chose, so that we can center our folum map on it
x = myeclipse.centroid.x.values[0]
y = myeclipse.centroid.y.values[0]
print('y: ' + str(y) + ', x: ' + str(x))

In [ ]:
map_stamen_water = folium.Map(
    location=[y,x],
    tiles='stamenwatercolor',
    zoom_start=3
)

folium.GeoJson(top_five,name='Eclipse Cities').add_to(map_stamen_water)
folium.GeoJson(myeclipse,name='Eclipse Path').add_to(map_stamen_water)

folium.LayerControl().add_to(map_stamen_water)

In [ ]:
map_stamen_water

In [ ]:
# You can write a function that creates a map with all of your properties
# This one creates popups for your cities

def plot_city_pop(top_five, myeclipse):
    # generate a new map
    folium_map = folium.Map(location=[y, x], zoom_start=3, tiles="stamenwatercolor")
    folium.GeoJson(myeclipse,name='Eclipse Path').add_to(folium_map)
    # for each row in the data, add a cicle marker
    for index, row in top_five.iterrows():
        
        # generate the popup message that is shown on click.
        popup_text = "Name: {}<br> Pop: {}"
        popup_text = popup_text.format(row["NAME"], row["POP"])
        
        folium.CircleMarker(location=
                            (row['geometry'].y, row['geometry'].x),
                            radius=6,popup=popup_text,fill=True).add_to(folium_map)        

    # Add a layer control if you like
    folium.LayerControl().add_to(folium_map)
    
    return folium_map

In [ ]:
# Call the function to create the map
plot_city_pop(top_five, myeclipse)

# Python, Jupyter, and Esri can work together!

### [Esri and Jupyter](https://notebooks.esri.com/user/vS9Rtz9xuzE8lsZGtZNXRQoPN/tree/samples)

# Thank you!